## This notebook contains the code to run my crossmatch, as well as refine the results to the desired radius (and other parameters)

In [1]:
import astropy.units as u
from astroquery.gaia import Gaia
from astropy.coordinates import SkyCoord, Angle
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table, vstack

import eliminating

In [3]:
# first step is to run get matches for 10 arcsec
filetouse = '/home/billee/Binary-Pulsar-Distances/Binary_Pulsar_Distances/atnf_psrcat.csv' #file updated as of 3/5/24
table, hits, skipped = eliminating.get_matches(filetouse, radius= 10.) #get_matches is modified to skip the first 2 lines of the file

# save as npz
filename = '/home/billee/Binary-Pulsar-Distances/Binary_Pulsar_Distances/matches_10arcsec_3-8'
np.savez(filename, table)

INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]


In [8]:
npz = np.load('/home/billee/Binary-Pulsar-Distances/Binary_Pulsar_Distances/matches_10arcsec_3-8.npz', allow_pickle=True)
table = Table(npz['arr_0'])

In [6]:
# SkyCoord for each pulsar and Gaia source, respectively
psr_pos = SkyCoord(table['Pulsar RA'], table['Pulsar DEC'], frame = 'icrs', unit =(u.hourangle, u.deg))
source_pos = SkyCoord(table['ra'], table['dec'], frame = 'icrs', unit= (u.deg, u.deg))

source_seps = []
first_time = True
# table of the sky separations for each pulsar and Gaia source pair
for k in range(len(table)):
    if first_time:
        source_seps.insert(0,psr_pos[k].separation(source_pos[k]))
        first_time = False
    else:
        source_seps.append(psr_pos[k].separation(source_pos[k]))

In [7]:
#append source_seps to table
table.add_column(source_seps, name='source_seps', index=14)

#save new npz file with sourceseps
np.savez('/home/billee/Binary-Pulsar-Distances/Binary_Pulsar_Distances/matches_10arcsec_3-8.npz', table)

ValueError: Cannot replace column 'source_seps'.  Use Table.replace_column() instead.

In [10]:
# take the full 10 arcsec table and limit to those with a separation 1.3 arcseconds
radius_condition = Angle(table['source_seps'], unit=u.deg).arcsec <= 1.3
table_1arcsec = table[radius_condition]

In [11]:
table_1arcsec.show_in_notebook()

idx,Companion Pulsar,Pulsar RA,Pulsar DEC,Binary,Binary Companion,solution_id,DESIGNATION,source_id,random_index,ref_epoch,ra,ra_error,dec,dec_error,source_seps,parallax,parallax_error,parallax_over_error,pm,pmra,pmra_error,pmdec,pmdec_error,ra_dec_corr,ra_parallax_corr,ra_pmra_corr,ra_pmdec_corr,dec_parallax_corr,dec_pmra_corr,dec_pmdec_corr,parallax_pmra_corr,parallax_pmdec_corr,pmra_pmdec_corr,astrometric_n_obs_al,astrometric_n_obs_ac,astrometric_n_good_obs_al,astrometric_n_bad_obs_al,astrometric_gof_al,astrometric_chi2_al,astrometric_excess_noise,astrometric_excess_noise_sig,astrometric_params_solved,astrometric_primary_flag,nu_eff_used_in_astrometry,pseudocolour,pseudocolour_error,ra_pseudocolour_corr,dec_pseudocolour_corr,parallax_pseudocolour_corr,pmra_pseudocolour_corr,pmdec_pseudocolour_corr,astrometric_matched_transits,visibility_periods_used,astrometric_sigma5d_max,matched_transits,new_matched_transits,matched_transits_removed,ipd_gof_harmonic_amplitude,ipd_gof_harmonic_phase,ipd_frac_multi_peak,ipd_frac_odd_win,ruwe,scan_direction_strength_k1,scan_direction_strength_k2,scan_direction_strength_k3,scan_direction_strength_k4,scan_direction_mean_k1,scan_direction_mean_k2,scan_direction_mean_k3,scan_direction_mean_k4,duplicated_source,phot_g_n_obs,phot_g_mean_flux,phot_g_mean_flux_error,phot_g_mean_flux_over_error,phot_g_mean_mag,phot_bp_n_obs,phot_bp_mean_flux,phot_bp_mean_flux_error,phot_bp_mean_flux_over_error,phot_bp_mean_mag,phot_rp_n_obs,phot_rp_mean_flux,phot_rp_mean_flux_error,phot_rp_mean_flux_over_error,phot_rp_mean_mag,phot_bp_rp_excess_factor,phot_bp_n_contaminated_transits,phot_bp_n_blended_transits,phot_rp_n_contaminated_transits,phot_rp_n_blended_transits,phot_proc_mode,bp_rp,bp_g,g_rp,radial_velocity,radial_velocity_error,rv_method_used,rv_nb_transits,rv_nb_deblended_transits,rv_visibility_periods_used,rv_expected_sig_to_noise,rv_renormalised_gof,rv_chisq_pvalue,rv_time_duration,rv_amplitude_robust,rv_template_teff,rv_template_logg,rv_template_fe_h,rv_atm_param_origin,vbroad,vbroad_error,vbroad_nb_transits,grvs_mag,grvs_mag_error,grvs_mag_nb_transits,rvs_spec_sig_to_noise,phot_variable_flag,l,b,ecl_lon,ecl_lat,in_qso_candidates,in_galaxy_candidates,non_single_star,has_xp_continuous,has_xp_sampled,has_rvs,has_epoch_photometry,has_epoch_rv,has_mcmc_gspphot,has_mcmc_msc,in_andromeda_survey,classprob_dsc_combmod_quasar,classprob_dsc_combmod_galaxy,classprob_dsc_combmod_star,teff_gspphot,teff_gspphot_lower,teff_gspphot_upper,logg_gspphot,logg_gspphot_lower,logg_gspphot_upper,mh_gspphot,mh_gspphot_lower,mh_gspphot_upper,distance_gspphot,distance_gspphot_lower,distance_gspphot_upper,azero_gspphot,azero_gspphot_lower,azero_gspphot_upper,ag_gspphot,ag_gspphot_lower,ag_gspphot_upper,ebpminrp_gspphot,ebpminrp_gspphot_lower,ebpminrp_gspphot_upper,libname_gspphot,dist
0,J0024-7204C,00:23:50.3546,-72:04:31.5048,*,*,1636148068921376768,Gaia DR3 4689638064248154240,4689638064248154240,411032629,2016.0,5.960403740672885,43.583965,-72.07517791487159,73.64408,0.00030156061858660206,nan,nan,nan,nan,nan,nan,nan,nan,0.39492482,nan,nan,nan,nan,nan,nan,nan,nan,nan,48,0,48,0,422.73224,1329263.2,171.86522,143333.5,3,False,nan,nan,nan,nan,nan,nan,nan,nan,6,5,31.53835,6,2,0,0.21980388,21.06157,16,16,nan,0.1538433,0.3281301,0.39616972,0.6914988,118.94729,-77.216866,27.165503,-43.950542,False,35,2731.6663082015093,37.71168,72.43555,17.096298,2,6078.366060343639,647.97955,9.38049,15.879075,2,6615.706590475014,111.37528,59.40013,15.196455,4.647007,0,2,0,2,0,0.68262005,-1.2172232,1.8998432,nan,nan,0,0,0,0,nan,nan,nan,nan,nan,nan,nan,nan,0,nan,nan,0,nan,nan,0,nan,NOT_AVAILABLE,305.9232491374621,-44.89244378597212,311.24173833761307,-62.33300248101193,False,True,0,False,False,False,False,False,False,False,False,0.07399622,0.9156617,0.0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,,0.00030689797890063903
1,J0024-7204D,00:24:13.88092,-72:04:43.8524,*,*,1636148068921376768,Gaia DR3 4689639301203685888,4

In [12]:
# take out all results that have nan values for the parallax, pmra, pmdec, or parallaxes that don't make sense
nan_parallax_cond = np.isnan(table_1arcsec['parallax']) == False
parallax_removed = table_1arcsec[nan_parallax_cond]

nan_pmra_cond = np.isnan(parallax_removed['pmra']) == False
pmra_removed = parallax_removed[nan_pmra_cond]

nan_pmdec_cond = np.isnan(pmra_removed['pmdec']) == False
pmdec_removed = pmra_removed[nan_pmdec_cond]

neg_parallax_cond = pmdec_removed['parallax'] + pmdec_removed['parallax_error'] > 0
final_table = pmdec_removed[neg_parallax_cond]

In [13]:
final_table.show_in_notebook()

idx,Companion Pulsar,Pulsar RA,Pulsar DEC,Binary,Binary Companion,solution_id,DESIGNATION,source_id,random_index,ref_epoch,ra,ra_error,dec,dec_error,source_seps,parallax,parallax_error,parallax_over_error,pm,pmra,pmra_error,pmdec,pmdec_error,ra_dec_corr,ra_parallax_corr,ra_pmra_corr,ra_pmdec_corr,dec_parallax_corr,dec_pmra_corr,dec_pmdec_corr,parallax_pmra_corr,parallax_pmdec_corr,pmra_pmdec_corr,astrometric_n_obs_al,astrometric_n_obs_ac,astrometric_n_good_obs_al,astrometric_n_bad_obs_al,astrometric_gof_al,astrometric_chi2_al,astrometric_excess_noise,astrometric_excess_noise_sig,astrometric_params_solved,astrometric_primary_flag,nu_eff_used_in_astrometry,pseudocolour,pseudocolour_error,ra_pseudocolour_corr,dec_pseudocolour_corr,parallax_pseudocolour_corr,pmra_pseudocolour_corr,pmdec_pseudocolour_corr,astrometric_matched_transits,visibility_periods_used,astrometric_sigma5d_max,matched_transits,new_matched_transits,matched_transits_removed,ipd_gof_harmonic_amplitude,ipd_gof_harmonic_phase,ipd_frac_multi_peak,ipd_frac_odd_win,ruwe,scan_direction_strength_k1,scan_direction_strength_k2,scan_direction_strength_k3,scan_direction_strength_k4,scan_direction_mean_k1,scan_direction_mean_k2,scan_direction_mean_k3,scan_direction_mean_k4,duplicated_source,phot_g_n_obs,phot_g_mean_flux,phot_g_mean_flux_error,phot_g_mean_flux_over_error,phot_g_mean_mag,phot_bp_n_obs,phot_bp_mean_flux,phot_bp_mean_flux_error,phot_bp_mean_flux_over_error,phot_bp_mean_mag,phot_rp_n_obs,phot_rp_mean_flux,phot_rp_mean_flux_error,phot_rp_mean_flux_over_error,phot_rp_mean_mag,phot_bp_rp_excess_factor,phot_bp_n_contaminated_transits,phot_bp_n_blended_transits,phot_rp_n_contaminated_transits,phot_rp_n_blended_transits,phot_proc_mode,bp_rp,bp_g,g_rp,radial_velocity,radial_velocity_error,rv_method_used,rv_nb_transits,rv_nb_deblended_transits,rv_visibility_periods_used,rv_expected_sig_to_noise,rv_renormalised_gof,rv_chisq_pvalue,rv_time_duration,rv_amplitude_robust,rv_template_teff,rv_template_logg,rv_template_fe_h,rv_atm_param_origin,vbroad,vbroad_error,vbroad_nb_transits,grvs_mag,grvs_mag_error,grvs_mag_nb_transits,rvs_spec_sig_to_noise,phot_variable_flag,l,b,ecl_lon,ecl_lat,in_qso_candidates,in_galaxy_candidates,non_single_star,has_xp_continuous,has_xp_sampled,has_rvs,has_epoch_photometry,has_epoch_rv,has_mcmc_gspphot,has_mcmc_msc,in_andromeda_survey,classprob_dsc_combmod_quasar,classprob_dsc_combmod_galaxy,classprob_dsc_combmod_star,teff_gspphot,teff_gspphot_lower,teff_gspphot_upper,logg_gspphot,logg_gspphot_lower,logg_gspphot_upper,mh_gspphot,mh_gspphot_lower,mh_gspphot_upper,distance_gspphot,distance_gspphot_lower,distance_gspphot_upper,azero_gspphot,azero_gspphot_lower,azero_gspphot_upper,ag_gspphot,ag_gspphot_lower,ag_gspphot_upper,ebpminrp_gspphot,ebpminrp_gspphot_lower,ebpminrp_gspphot_upper,libname_gspphot,dist
0,J0024-7204D,00:24:13.88092,-72:04:43.8524,*,*,1636148068921376768,Gaia DR3 4689639301203685888,4689639301203685888,1470788328,2016.0,6.0567281274352,0.450227,-72.07879707725034,0.5400652,0.0003450226381780569,0.6099366817464894,0.47152585,1.2935381,13.508225,6.673575645513973,0.7774188,-11.744596498540092,0.6485255,0.12700611,0.22986352,-0.5704371,-0.0952923,-0.0707237,-0.40347257,0.66239357,-0.4505167,-0.15693362,-0.21463975,128,0,127,1,47.013535,58586.363,2.8129838,2530.568,95,False,nan,2.0127378,0.096017994,-0.048260015,-0.25239655,-0.15251344,0.2004025,-0.12796283,15,11,1.1908531,19,14,1,0.074234866,22.879084,56,0,5.2297726,0.022888836,0.046741795,0.07771128,0.3135696,58.303658,-49.785294,-9.0233,-22.060217,False,93,60058.35285875391,551.4077,108.91823,13.740933,8,61239.93101461995,2207.7295,27.738874,13.370955,8,86872.43949207109,2447.8555,35.489204,12.40069,2.466141,0,8,0,8,0,0.9702654,-0.36997795,1.3402433,nan,nan,0,0,0,0,nan,nan,nan,nan,nan,nan,nan,nan,0,nan,nan,0,nan,nan,0,nan,NOT_AVAILABLE,305.8811130484171,-44.89330769056964,311.26849387391906,-62.36016105661894,False,False,0,False,False,False,False,False,False,True,False,2.4344329e-09,1.88